# UNET-Ensemble
- UNET의 구조에 착안하여 만든 모델
- UNET의 구조처럼 잠재벡터를 단계적으로 만들어 나가면서, 단계별로 잠재벡터를 형성한다.
- UNET의 최하단에 진입했을 때, 다시 상단으로 올라오면서 예측값을 생성한다.
- 이 때, 상단으로 올라오는 단계별로 하단에서 만든 잠재벡터들을 아래에서 위로 끌어오면서, 모델이 하단으로 가면서 잃어버렸던 정보량을 최대한 보전하려고 노력한다.
- 이 후, 최 상단에서는 최 하단으로 진입하는 단계에서 단계적으로 생성한 잠재벡터들을 모두 고려한 결과값을 도출한다.

### 패키지 로딩

In [1]:
# Data Handling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt

# To ignore warnings
import warnings 
warnings.filterwarnings('ignore')

# Seed 고정
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) 

# Keras and Tensorflow
import keras
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,Dropout,BatchNormalization,Add,Input,Activation,Concatenate,Average
from keras.layers.merge import concatenate

# Scoring
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.model_selection import KFold

### 데이터 로딩

In [2]:
### 데이터 로딩
X = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/X_train.csv")
y = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/y_train.csv")

### 딥러닝 학습을 위한 데이터 넘파이 변환
X = np.array(X)
y = np.array(y)

### 모델링

In [4]:
### 모델 구축

# input 정의
input_layer = keras.layers.Input(shape=(56,), dtype="float32")

# go to deep
go1 = keras.layers.Dense(32, activation="relu",kernel_initializer="he_normal")(input_layer)
go2 = keras.layers.Dense(32, activation="relu",kernel_initializer="he_normal")(go1)
batch1 = keras.layers.BatchNormalization()(go2)
go3 = keras.layers.Dense(64, activation="relu",kernel_initializer="he_normal")(batch1)
go4 = keras.layers.Dense(64, activation="relu",kernel_initializer="he_normal")(go3)
batch2 = keras.layers.BatchNormalization()(go4)
go5 = keras.layers.Dense(128, activation="relu",kernel_initializer="he_normal")(batch2)
go6 = keras.layers.Dense(128, activation="relu",kernel_initializer="he_normal")(go5)
batch3 = keras.layers.BatchNormalization()(go6)
go7 = keras.layers.Dense(256, activation="relu",kernel_initializer="he_normal")(batch3)
go8 = keras.layers.Dense(256, activation="relu",kernel_initializer="he_normal")(go7)
batch4 = keras.layers.BatchNormalization()(go8)
go9 = keras.layers.Dense(512, activation="relu",kernel_initializer="he_normal")(batch4)
go10 = keras.layers.Dense(512, activation="relu",kernel_initializer="he_normal")(go9)
batch5 = keras.layers.BatchNormalization()(go10)

# go to up
up1 = keras.layers.Dense(256, activation="relu",kernel_initializer="he_normal")(batch5)
up2 = keras.layers.Dense(256, activation="relu",kernel_initializer="he_normal")(up1)
concat1 = keras.layers.Concatenate()([go8 ,up2])
up3 = keras.layers.Dense(128, activation="relu",kernel_initializer="he_normal")(concat1)
up4 = keras.layers.Dense(128, activation="relu",kernel_initializer="he_normal")(up3)
concat2 = keras.layers.Concatenate()([go6, up4])
up5 = keras.layers.Dense(64, activation="relu",kernel_initializer="he_normal")(concat2)
up6 = keras.layers.Dense(64, activation="relu",kernel_initializer="he_normal")(up5)
concat3 = keras.layers.Concatenate()([go4, up6])
up7 = keras.layers.Dense(32, activation="relu",kernel_initializer="he_normal")(concat3)
up8 = keras.layers.Dense(32, activation="relu",kernel_initializer="he_normal")(up7)
concat4 = keras.layers.Concatenate()([go2, up8])

# result
result_layer = keras.layers.Dense(14)(concat4)

In [5]:
### 모델 Fitting 함수 정의
def fit_model(X_train,X_val,y_train,y_val,i): 
    model = keras.Model(
        inputs=input_layer,
        outputs=result_layer)
    model.compile(loss='mse', optimizer='adam')

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath = 'C:/LG_Aimers/U_NET_ALL/model/'+'fold' + str(i) + '--.{val_loss:.3f}'+'.h5', 
        monitor='val_loss', 
        save_best_only=True)
    model.fit(X_train, 
              y_train,
              batch_size=32,
              epochs=300, 
              validation_data=(X_val, y_val), 
              callbacks=[early_stopping, checkpoint])
    return model

In [ ]:
### 모델 학습
kf=KFold(10,shuffle=True)
i=0
for train_index,val_index in kf.split(X):
  i+=1
  X_train,X_val=X[train_index],X[val_index]
  y_train,y_val=y[train_index],y[val_index]
  model=fit_model(X_train,X_val,y_train,y_val,i)
  print('finish {} model'.format(i))

In [2]:
### KFOLD 10 Model 불러오기
model1=load_model("C:/LG_Aimers/U_NET_ALL/model/fold1--.1.526.h5")
model2=load_model("C:/LG_Aimers/DENSE_CHAIN/model/fold2--.1.632.h5")
model3=load_model("C:/LG_Aimers/U_NET_ALL/model/fold3--.1.742.h5")
model4=load_model("C:/LG_Aimers/U_NET_ALL/model/fold4--.1.528.h5")
model5=load_model("C:/LG_Aimers/U_NET_ALL/model/fold5--.1.431.h5")
model6=load_model("C:/LG_Aimers/U_NET_ALL/model/fold6--.1.720.h5")
model7=load_model("C:/LG_Aimers/U_NET_ALL/model/fold7--.1.479.h5")
model8=load_model("C:/LG_Aimers/U_NET_ALL/model/fold8--.1.574.h5")
model9=load_model("C:/LG_Aimers/U_NET_ALL/model/fold9--.1.507.h5")
model10=load_model("C:/LG_Aimers/U_NET_ALL/model/fold10--.1.513.h5")

In [3]:
### 앙상블 모델 형성
all_models=list([model1,model3,model4,model5,model6,model7,model8,model9,model10])

### 앙상블 모델 정의 함수 구축
def define_stacked_model(members):
  # update all layers in all models to not be trainiable
  for i in range(len(members)):
    model=members[i]
    for layer in model.layers:
      layer.trainable=False
      # rename to avoid 'unique layer name' issue
      layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
  
  # define multi-handed input
  ensemble_visible=[model.input for model in members]
  ensemble_outputs=[model.output for model in members]

  y=Average()(ensemble_outputs)

  model=Model(inputs=ensemble_visible,outputs=y,name='ensemble')

  keras.utils.plot_model(model,show_shapes=True,to_file='C:\LG_Aimers\XGB노가다\Y_03_결과\model_graph.jpg')

  model.compile(loss='mae',optimizer='adam')
  return model

### 앙상블 모델 Fitting 함수 구축
def fit_stacked_model(model,trainX,valX,trainY,valY):
  x_train=[trainX for _ in range(len(model.input))]
  x_val=[valX for _ in range(len(model.input))]
  y_train=trainY
  y_val=valY

  es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)
  checkpoint_filepath='/content/drive/MyDrive/lg aimers/ensemble_model.h5'
  cp=keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor='val_loss',
      save_best_only=True
  )
  model.fit(x_train,y_train,epochs=1,validation_data=(x_val,y_val),callbacks=[es,cp],batch_size=32)


### 앙상블모델 평가함수 구축
def evaluate_stacked_model(model,inputX,y_val):
  x_val=[inputX for _ in range(len(model.input))]
  return model.evaluate(x_val,y_val)

### 앙상블모델 예측함수 구축
def predict_stacked_model(model,inputX):
  x_test=[inputX for _ in range(len(model.input))]
  return model.predict(x_test)

In [ ]:
### 앙상블모델 학습 및 평가
n_members=10
members=all_models

stacked_model=define_stacked_model(members)
fit_stacked_model(stacked_model,X_train,X_val,y_train,y_val)

evaluate_stacked_model(stacked_model,X_val,y_val)

### 예측

In [5]:
### 데이터 로딩
X_valid = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/X_valid.csv")
y_valid = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/y_valid.csv")

### 앙상블 모델 예측
result=predict_stacked_model(stacked_model,X_valid)

### 예측 결과 넘파이 변환
y_valid_array = np.array(y_valid)

In [6]:
### 평가함수 구축
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [7]:
### 스코어 계산
lg_nrmse(y_valid_array, result)